In [1]:
!nvidia-smi

Fri Oct  8 07:54:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-lightning
!pip install hydra-core --upgrade
!pip install wandb
!pip install pytorch_metric_learning
!pip install -U albumentations
!pip install torchmetrics

     |████████████████████████████████| 925 kB 14.2 MB/s 
     |████████████████████████████████| 125 kB 33.9 MB/s 
     |████████████████████████████████| 829 kB 64.3 MB/s 
     |████████████████████████████████| 282 kB 64.4 MB/s 
     |████████████████████████████████| 636 kB 75.1 MB/s 
     |████████████████████████████████| 1.3 MB 59.7 MB/s 
     |████████████████████████████████| 160 kB 73.1 MB/s 
     |████████████████████████████████| 271 kB 58.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=22a027da1638445a3bdc3d1d44a41dd2cb4c11720c3e5b2feb550793b04d8572
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-

     |████████████████████████████████| 1.7 MB 14.6 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 180 kB 61.1 MB/s 
     |████████████████████████████████| 139 kB 61.2 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=11fdb4ac0c98a01d0689038874261f1038e904c953a30c08be4822ee198a3d21
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=c70a20c746fea607d51a50cc128092ad66f5fcd972a5191063b33d691ec1168d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 105 kB 15.3 MB/s 
     |████████████████████████████████| 102 kB 14.8 MB/s 
     |████████████████████

In [3]:
cd drive/MyDrive/kaggle/petfinder

/content/drive/MyDrive/kaggle/petfinder


In [104]:
import os
import sys
from glob import glob

import warnings
import argparse
import random
import gc
import pandas as pd
from glob import glob
from tqdm import tqdm
import numpy as np
import torchvision
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

import pandas as pd
import numpy as np

import cv2

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, random_split, Dataset

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from split_cv import get_cv
import lightgbm as lgb

sys.path.append('models')
import dino_model as vits

warnings.simplefilter('ignore')

In [7]:
def create_dino_model(path=None):
    model = vits.__dict__["vit_small"](patch_size=16, num_classes=0)

    if os.path.isfile(path):
        state_dict = torch.load(path)
        state_dict = state_dict['teacher']

    # remove `module.` prefix
    state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    # remove `backbone.` prefix induced by multicrop wrapper
    state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict, strict=False)
    return model

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = create_dino_model("/content/drive/MyDrive/kaggle/petfinder/dino/dino_weights_1024/checkpoint0100.pth").to(device)

In [30]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score/test.csv')

train_dir = '/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score/train'
test_dir = '/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score/test'

In [31]:
def get_train_file_path(image_id):
  return "{}/{}.jpg".format(train_dir, image_id)

def get_test_file_path(image_id):
  return "{}/{}.jpg".format(test_dir, image_id)

train['file_path'] = train['Id'].apply(get_train_file_path)
test['file_path'] = test['Id'].apply(get_test_file_path)  

In [71]:
def transforms():
    return A.Compose([
                      A.Resize(224, 224),
                      A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                      ToTensorV2()])

In [83]:
transform = transforms()
feature_list = []

for i in tqdm(range(len(train))):
  row = train.loc[i]
  image = cv2.imread(row["file_path"])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = transform(image=image)['image'].float()
  image = image.unsqueeze(0).to(device)
  feature = model(image)
  feature = torch.squeeze(feature).detach().cpu().numpy()
  feature_list.append(feature)

100%|██████████| 9912/9912 [03:11<00:00, 51.77it/s]


In [84]:
feature_list = np.array(feature_list)
np.save('feature.npy', feature_list)

In [88]:
for i in range(feature_list.shape[1]):
  train[f"dino_{i}"] = feature_list[:, i]

In [90]:
train = train.drop(["file_path", "Id"], axis=1)

In [95]:
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(train, train["Pawpularity"])):
  train.loc[val_index, 'fold'] = int(n)

In [96]:
# example of parameters
lgbm_params = {
    'objective': 'rmse', # Binary classification : 2値分類ではこれを使う
    'seed': 42, # random seed : これを固定すると、再現性が出る
    'metric': 'rmse', 
    'learning_rate': 0.01,
    'max_bin': 800, # depth
    'num_leaves': 80, # leaves,
    "verbose":-1
}

In [97]:
from sklearn.metrics import mean_squared_error

def rmsescore(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [100]:
features = [i for i in train.columns if i not in ["Pawpularity", "fold"]]
target = "Pawpularity"

In [105]:
scores = []
allpreds = []

for fold in range(5):
    p_train = train[train["fold"] != fold]
    p_val = train[train["fold"] == fold]

    p_train = p_train.reset_index(drop=True)
    p_val = p_val.reset_index(drop=True)

    lgb_train = lgb.Dataset(p_train[features], p_train[target])
    lgb_eval = lgb.Dataset(p_val[features], p_val[target])

    model = lgb.train(
        lgbm_params,
        lgb_train,
        valid_sets=lgb_eval,
        verbose_eval=50,
        num_boost_round=1000,
        early_stopping_rounds=100)
                                 

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 20.3941
[100]	valid_0's rmse: 20.2995
[150]	valid_0's rmse: 20.2249
[200]	valid_0's rmse: 20.1861
[250]	valid_0's rmse: 20.1669
[300]	valid_0's rmse: 20.1729
[350]	valid_0's rmse: 20.1687
Early stopping, best iteration is:
[276]	valid_0's rmse: 20.1597
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 20.3253
[100]	valid_0's rmse: 20.1944
[150]	valid_0's rmse: 20.1357
[200]	valid_0's rmse: 20.0822
[250]	valid_0's rmse: 20.0489
[300]	valid_0's rmse: 20.022
[350]	valid_0's rmse: 20.0083
[400]	valid_0's rmse: 20.013
[450]	valid_0's rmse: 20.0198
Early stopping, best iteration is:
[364]	valid_0's rmse: 20.005
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 20.4273
[100]	valid_0's rmse: 20.2993
[150]	valid_0's rmse: 20.1634
[200]	valid_0's rmse: 20.1158
[250]	valid_0's rmse: 20.094
[300]	valid_0's rmse: 20.0514
[350]	valid_0's rmse: 20.013

In [ ]:
feature_importances = model.feature_importance()